## This example shows how we can interact with AGOL using the ArcGIS API for Python to connect to a GIS, query a web layer's data, map it in the Jupyter notebook and download it to a Shapefile

#### Some Resources on the API: 
##### https://developers.arcgis.com/python/latest/
##### https://developers.arcgis.com/python/latest/guide/overview-of-the-arcgis-api-for-python/

<div><img src="https://developers.arcgis.com/python/latest/guide/images/guide_api_modules_overview.png" style="width: 800px;"/></div>

In [ ]:
# Start with your Imports
import os
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection, Feature, FeatureSet

## Connect to the GIS

In [ ]:
# Example how to connect to AGOL without credentials (for items shared to Public)
# Reference: https://developers.arcgis.com/python/latest/guide/working-with-different-authentication-schemes/
gis = GIS()

## Query fire station facility layer

#### This is a USGS publically hosted feature layer we are looking to use today: 
##### https://carto.nationalmap.gov/arcgis/rest/services/structures/MapServer/16
##### It was originally found on AZGEO, but we wanted to go to the source.  
##### https://azgeo-open-data-agic.hub.arcgis.com/datasets/b0aaa5d6f376467884cab24f7139a2bf_51/explore?location=33.325092%2C-111.838967%2C10.67

In [ ]:
# Can get layers by Item ID
# example call 
# https://services2.arcgis.com/FiaPA4ga0iQKduv3/ArcGIS/rest/services/Structures_Medical_Emergency_Response_v1/FeatureServer
flc = gis.content.get("2c36dbb008844081b017da6fd3d0d28b")
print(flc)

# Can also capture by URL
#flc = FeatureLayerCollection('https://carto.nationalmap.gov/arcgis/rest/services/structures/MapServer', gis)
#print(flc)

In [ ]:
# How you can inspect the layers of a Service
for lyr in flc.layers:
    print(lyr.properties.name +','  +  str(lyr.properties.id))

In [ ]:
# Select Layer by position in list, not necessarily by ID number
stationFL = flc.layers[2]
#stationFL = flc.layers[51]

# Verify you chose the correct one
print(stationFL)

In [ ]:
# Query the feature by City of Choice, i.e. Tempe
# Reference for syntax: https://developers.arcgis.com/python/latest/guide/tutorials/query-a-feature-layer-sql/
where_clause = "City = 'Tempe'"
queryResult = stationFL.query(
    where = where_clause, 
    out_fields=["Name", "Address","CITY","STATE"],
    return_Geometry=True,
    out_sr=4326
)

# Convert the query results to features
# Reference: https://developers.arcgis.com/python/latest/api-reference/arcgis.features.toc.html
stationFeatures = queryResult.features
for feature in stationFeatures:
    print(feature.attributes)

In [ ]:
# Convert the feature results to a featureset
stationFS = FeatureSet(stationFeatures)

## Display the fire stations on a map

In [ ]:
# In ArcGIS API For Python Version older than 2.4, you can create a map widget that can display right in your Jupyter Notebook to visualize your results
# https://developers.arcgis.com/python-2-3/api-reference/arcgis.widgets.html
#map1 = gis.map('Tempe')
#map1.add_layer(stationFS)
## You can specify a zoom level
#map1.zoom = 12
#map1.zoom_to_layer(stationFS)
#map1

#  At version 2.4 and later there is a different appraoch.  This starts at Pro 3.5 version
# https://developers.arcgis.com/python/latest/api-reference/arcgis.gis.toc.html#arcgis.gis.GIS.map
map2  = gis.map('Tempe')
map2.content.add(stationFS)
map2.zoom = 10
#map2.zoom_to_layer(stationFS)
map2

## Export your results to a Shapefile

In [ ]:
# Use Pro Projects location to determine output location for Shapefile
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprxPath = aprx.filePath
print(aprxPath)

# Get the folder of the current Pro Project
projPath = os.path.dirname(aprxPath)
print(projPath)

In [ ]:
# Save Stations to Shapefile
stationFS.save(projPath, 'TempeStations')